# Silver Layer - Clean and Structure the Data

🎯 **Goal**: Make the data clean, consistent, and usable for downstream tasks

🔧 **Tasks a data engineer performs**:
- Read from the bronze layer
- Flatten nested structures (e.g. explode arrays)
- Drop invalid or duplicate rows
- Convert data types to the correct formats (e.g., height as float)
- Rename columns for clarity or consistency
- Store result as a new dataset

### 🧹 This notebook: Silver Layer – Cleaned & Structured Pokémon Data

In this notebook, we:

- Read the raw Pokémon data from the bronze Delta table
- **Explode** the `types` array into individual rows (so each type has its own row)
- Drop the original nested `types` column
- Ensure data types are consistent (e.g. `height` as float)
- Rename columns if needed for clarity
- Save the cleaned data to a new Delta table in the **silver layer** at `./data/silver/pokemon`

🥈 This layer prepares the data for analysis and ML by enforcing structure and consistency.
